In [284]:
from sklearn.metrics.pairwise import cosine_similarity
import nltk

nltk.download('stopwords')
import numpy as np
from nltk.stem import LancasterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import os

lancStemmer = LancasterStemmer()  # stemming algorithm Lancaster


# lemmatizer = WordNetLemmatizer() #lemmatizer algorithm

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sergiogonzalez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [285]:
def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    palabras = [palabra.lower() for palabra in re.findall(r'\w+', text.lower())]
    text_lista = []
    for palabra in palabras:
        if palabra not in stopwords:
            text_lista.append(palabra)
    nuevo_texto = ' '.join(text_lista)
    return nuevo_texto

In [286]:

def get_stemmer(text):
    palabras = remove_stopwords(text)
    palabras = palabras.split()
    text_lista = []
    for palabra in palabras:
        nueva = lancStemmer.stem(palabra)
        text_lista.append(nueva)
    nuevo_texto = ' '.join(text_lista)
    return nuevo_texto

In [287]:

def get_grams(text, n):
    text = get_stemmer(text)  # pre-procesa el parrafo
    text = text.split()  # separa los caracteres pre-procesados del parrafo en listas
    if n == 0:
        return text
    else:
        grams = ngrams(text, n)  # genera los ngrams
        result = []
        for ng in grams:
            result.append(' '.join(ng))  # agrega los ngrams en una lista llamada result
        return result

In [288]:

def pre_procesados (folder_path, n):
  preprocess_texts = []
  for fileid in os.listdir(folder_path):
    if fileid.endswith(".txt"):
      filepath = os.path.join(folder_path, fileid)
      with open(filepath, 'r', encoding='latin1', errors='ignore') as file:
        text = file.read()
        grams = get_grams(text, n)
        preprocess_texts.append((fileid, grams))
  return preprocess_texts

In [289]:
def matriz_parrafos(grams1, grams2):
    grams_palabras = set(grams1 + grams2)  # set de palabras de ambos ngrams
    grams_juntos = [grams1, grams2]  # lista con ambas listas de los ngrams de cada parrafo
    matriz = []
    for grama in grams_juntos:
        vector = []
        for palabra in grams_palabras:
            vector.append(
                1 if palabra in grama else 0)  # compara las palabras de los grams a la palabra y agrega 1 o 0 al vector del parrafo
        matriz.append(vector)
    return matriz

In [290]:
# Obtener n-gramas preprocesados
folder_path = "/Users/sergiogonzalez/Documents/GitHub/DetectorPlagio/textos_plagiados" # Ruta de la carpeta con los textos plagiados
preprocess_plagiados = pre_procesados(folder_path, 3)

folder_path_og = "/Users/sergiogonzalez/Documents/GitHub/DetectorPlagio/docs_originales" # Ruta de la carpeta con los textos originales
preprocess_originales = pre_procesados(folder_path_og, 3)

for id_plagiado, (name_plagiado, grams_plagiado) in enumerate(preprocess_plagiados, 1):
    print(f'\nDocumento analizado: {name_plagiado}')
    for id_original, (name_original, grams_original) in enumerate(preprocess_originales, 1):
        similitud = cosine_similarity(matriz_parrafos(grams_plagiado, grams_original))
        print(f"Similitud de Coseno entre {name_plagiado} y {name_original}: {similitud[0][1]}")


Documento analizado: FID-06.txt
Similitud de Coseno entre FID-06.txt y org-107.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-098.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-067.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-073.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-072.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-066.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-099.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-106.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-104.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-110.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-058.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-070.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-064.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-065.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-071.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-059.txt: 0.0
Similitud de Coseno entre FID-06.txt y org-105.txt: 0.0
Similitud de Co

### LAB CADENAS DE OCURRENCIA

In [291]:
resultados = []
for id_plagiado, (name_plagiado, grams_plagiado) in enumerate(preprocess_plagiados, 1):
    for id_original, (name_original, grams_original) in enumerate(preprocess_originales, 1):
        similitud = cosine_similarity(matriz_parrafos(grams_plagiado, grams_original))
        if similitud[0][1] != 0.0 and similitud[0][1] >= 0.2:
            resultados.append([name_plagiado, name_original, similitud[0][1]])
            

In [292]:
resultados.sort(key=lambda x: x[2], reverse=True)
resultados

[['FID-05.txt', 'org-085.txt', 0.9999999999999983],
 ['FID-03.txt', 'org-016.txt', 0.9173191625339879],
 ['FID-04.txt', 'org-045.txt', 0.9055118110236218],
 ['FID-08.txt', 'org-079.txt', 0.8382349847675461],
 ['FID-09.txt', 'org-109.txt', 0.808248929265168],
 ['FID-02.txt', 'org-104.txt', 0.6631210306331439],
 ['FID-01.txt', 'org-076.txt', 0.6468162120117367],
 ['FID-07.txt', 'org-041.txt', 0.5918092276655763],
 ['FID-10.txt', 'org-007.txt', 0.5786876586795143],
 ['FID-06.txt', 'org-043.txt', 0.55745196041636]]

### Resultados Esperados
1-. una lista ordenada,descendentemente por similitud, donde cada entrada incluye, para cada documento asociado.

2-. puntaje de similitud.

3-. cadena de coincidencia.

4-. longitud y el apuntador que nos lleva a su ocurrencia.

In [293]:
resultados.sort(key=lambda x: x[2], reverse=True)
lista_titulos = []
for resultado in resultados:
    for plagiados in preprocess_plagiados:
        if plagiados[0] == resultado[0]:
            lista_titulos.append([plagiados[0], resultado[1]])
            

In [294]:
lista_titulos

[['FID-05.txt', 'org-085.txt'],
 ['FID-03.txt', 'org-016.txt'],
 ['FID-04.txt', 'org-045.txt'],
 ['FID-08.txt', 'org-079.txt'],
 ['FID-09.txt', 'org-109.txt'],
 ['FID-02.txt', 'org-104.txt'],
 ['FID-01.txt', 'org-076.txt'],
 ['FID-07.txt', 'org-041.txt'],
 ['FID-10.txt', 'org-007.txt'],
 ['FID-06.txt', 'org-043.txt']]

In [295]:
import os
import nltk

def buscar_y_tokenizar(directorio, nombre_archivo):
    for filename in os.listdir(directorio):
        if filename == nombre_archivo:
            filepath = os.path.join(directorio, filename)
            with open(filepath, 'r', encoding='latin1', errors='ignore') as file:
                text = file.read()
                sentences = nltk.sent_tokenize(text)
                return sentences
    return None

In [296]:
import difflib

def encontrar_coincidencias(sentences_originales, sentences_plagiados):
    coincidencias = []

    for sentence_orig in sentences_originales:
        for sentence_plag in sentences_plagiados:
            matcher = difflib.SequenceMatcher(None, sentence_orig, sentence_plag)
            match = matcher.find_longest_match(0, len(sentence_orig), 0, len(sentence_plag))
            if match.size > 0:
                # Aplicar stemming y eliminar stopwords a las coincidencias antes de contar las palabras
                cadena_orig_stemmed = get_stemmer(sentence_orig[match.a:match.a + match.size])
                cadena_plag_stemmed = get_stemmer(sentence_plag[match.b:match.b + match.size])
                # Contar las palabras en las coincidencias después de aplicar el stemming y eliminar las stopwords
                palabras_orig = remove_stopwords(cadena_orig_stemmed).split()
                palabras_plag = remove_stopwords(cadena_plag_stemmed).split()
                if len(palabras_orig) > 1 and len(palabras_plag) > 1:  # Solo considerar coincidencias con más de una palabra
                    coincidencias.append({
                        "cadena_orig": sentence_orig[match.a:match.a + match.size],
                        "cadena_plag": sentence_plag[match.b:match.b + match.size],
                        "longitud": match.size
                    })

    return coincidencias


In [297]:
def calcular_similitud_ngramas(sentences_originales, sentences_plagiados, n):
    grams_originales = []
    grams_plagiados = []

    for sentence in sentences_originales:
        grams_originales.extend(get_grams(sentence, n))
    for sentence in sentences_plagiados:
        grams_plagiados.extend(get_grams(sentence, n))
        

    matriz = matriz_parrafos(grams_originales, grams_plagiados)
    # Calcular la similitud coseno entre las matrices de n-gramas
    similitud = cosine_similarity(matriz)[0][1]

    return similitud

In [312]:
for titulo in lista_titulos:
    resultados = []
    sentences_originales = buscar_y_tokenizar(folder_path_og, titulo[1])
    sentences_plagiados = buscar_y_tokenizar(folder_path, titulo[0])
    print(f"Titulo: {titulo[0]}")
    
    if sentences_originales and sentences_plagiados:
        similitud = calcular_similitud_ngramas(sentences_originales, sentences_plagiados, 0) 
        print(f"Similitud entre '{titulo[0]}' y '{titulo[1]}': {similitud * 100:.2f}%")
        coincidencias = encontrar_coincidencias(sentences_originales, sentences_plagiados)
        print(f"Coincidencias para '{titulo[0]}' y '{titulo[1]}':")
        
        for coincidencia in coincidencias:
            print(f"Cadena original: {coincidencia['cadena_orig']} (Longitud: {coincidencia['longitud']})")
            print(f"Cadena plagiada: {coincidencia['cadena_plag']}")
            print()
    else:
        print(f"No se encontraron oraciones en '{titulo[0]}' o '{titulo[1]}'")
    print("----------------------------\n")


Titulo: FID-05.txt
Similitud entre 'FID-05.txt' y 'org-085.txt': 100.00%
Coincidencias para 'FID-05.txt' y 'org-085.txt':
Cadena original: Internet of Things (IoT) based remote healthcare applications provide (Longitud: 69)
Cadena plagiada: Internet of Things (IoT) based remote healthcare applications provide

Cadena original: e application (Longitud: 13)
Cadena plagiada: e application

Cadena original: s a complex task and diagnosis results  (Longitud: 39)
Cadena plagiada: s a complex task and diagnosis results 

Cadena original: g heart disease  (Longitud: 16)
Cadena plagiada: g heart disease 

Cadena original: To address this issue, a novel Recommendation System for Cardiovascular Disease Prediction Using IoT Network (DEEP-CARDIO) has been proposed for providing prior diagnosis, treatment, and dietary recommendations for cardiac diseases. (Longitud: 232)
Cadena plagiada: To address this issue, a novel Recommendation System for Cardiovascular Disease Prediction Using IoT Network (DEE

In [314]:
!pip install fpdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 335.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 9.9 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: fonttools
    Found existing installation: fonttools 4.25.0
    Uninstalling fonttools-4.25.0:
      Successfully uninstalled fonttools-4.25.0


In [318]:
def crear_documento_pdf(titulo, similitud, coincidencias):
    # Crear un nuevo objeto PDF
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Agregar una página
    pdf.add_page()

    # Establecer la fuente y el tamaño del texto
    pdf.set_font("Arial", size=12)

    # Título del documento
    pdf.cell(200, 10, txt=f"Resultados de prueba de plagio: {titulo[1]}", ln=True, align="C")

    # Plagio detectado
    pdf.cell(200, 10, txt=f"Plagio detectado: {similitud * 100:.2f}%", ln=True, align="C")
    pdf.ln(5)

    # Texto con resaltado de las coincidencias
    for coincidencia in coincidencias:
        cadena_orig = coincidencia['cadena_orig']
        cadena_plag = coincidencia['cadena_plag']
        # Verificar si existe la clave 'referencia'
        if 'referencia' in coincidencia:
            referencia = coincidencia['referencia']
            texto = f"Texto original: {cadena_orig}\nTexto plagiado: {cadena_plag}\nReferencia: {referencia}\n\n"
        else:
            texto = f"Texto original: {cadena_orig}\nTexto plagiado: {cadena_plag}\n\n"
        pdf.set_font("Arial", style="B")
        pdf.multi_cell(0, 10, txt=texto, border=1, align="L")

    # Guardar el documento PDF en la carpeta de resultados
    nombre_archivo = f"Resultado_similitud_{titulo[0]}_y_{titulo[1]}.pdf"
    ruta_archivo = os.path.join("/Users/sergiogonzalez/Documents/GitHub/DetectorPlagio/app/Resultados", nombre_archivo)
    pdf.output(ruta_archivo)


In [319]:
for titulo in lista_titulos:
    resultados = []
    sentences_originales = buscar_y_tokenizar(folder_path_og, titulo[1])
    sentences_plagiados = buscar_y_tokenizar(folder_path, titulo[0])

    if sentences_originales and sentences_plagiados:
        similitud = calcular_similitud_ngramas(sentences_originales, sentences_plagiados, 0) 
        coincidencias = encontrar_coincidencias(sentences_originales, sentences_plagiados)
            
        # Llamar a la función para crear el documento PDF
        crear_documento_pdf(titulo, similitud, coincidencias)

/var/folders/y1/nsw2h6qd4l3407yyy_3z_xx00000gn/T/ipykernel_93623/2745752084.py:10: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
/var/folders/y1/nsw2h6qd4l3407yyy_3z_xx00000gn/T/ipykernel_93623/2745752084.py:13: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt=f"Resultados de prueba de plagio: {titulo[1]}", ln=True, align="C")
/var/folders/y1/nsw2h6qd4l3407yyy_3z_xx00000gn/T/ipykernel_93623/2745752084.py:13: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt=f"Resultados de prueba de plagio: {titulo[1]}", ln=True, align="C")
/var/folders/y1/nsw2h6qd4l3407yyy_3z_xx00000gn/T/ipykernel_93623/2745752084.py:16: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt=f"Plagio detectado: {